In [1]:
%load_ext autoreload
%autoreload 2
import time
import tensorflow as tf
import p8_util

Using TensorFlow backend.


In [2]:
OUTPUT_DIR = './tmp/baseline'
OUTPUT_DIR_TB = './tmp'
datadir = './data'
is_tensorboard = True

In [19]:
# this will kill the processes for Tensorboard
#is_tensorboard = True
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill
# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill    

kill: (32577)kill: (32583)

In [4]:
import p8_util
filename_dataset=datadir+'/arr_keras_X_y_train_test.dump'
x_train, x_test, y_train, y_test, nClasses, tuple_dimension = p8_util.load_dataset(filename_dataset)

nClasses, tuple_dimension

p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump
(414, 224, 224, 3) (47, 224, 224, 3) (414, 3) (47, 3)
(414, 224, 672) (47, 224, 672) (414, 3) (47, 3)
(414, 672, 224) (47, 672, 224) (414, 3) (47, 3)


(3, (672, 224))

#### Increase of train dataset against test dataset.

In [5]:
import p8_util_config
import NNAdaNetBuilder

#-----------------------------------------------------------------
# Get the number of convolutional layers for CNN network
# This will fixe the equivalent parameter of AdaNet num layers.
#-----------------------------------------------------------------
if p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type'] == 'CNN' :
    layer_num = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_layer_config']['conv_layer_num']
elif p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_type'] == 'RNN' :
    layer_num = p8_util_config.dict_adanet_config['adanet_nn_layer_config']['nn_layer_config']['rnn_layer_num']

print("\n Number of convolutional layers= {}".format(layer_num))
oNNAdaNetBuilder = NNAdaNetBuilder.NNAdaNetBuilder(p8_util_config.dict_adanet_config, num_layers=layer_num)

oNNAdaNetBuilder.show()

print("\nMax steps= {} / Number of EPOCH={}".format(p8_util_config.MAX_STEPS,p8_util_config.NUM_EPOCHS))


 Number of convolutional layers= 2

*** NNAdaNetBuilder : NN Type=RNN


NN type              : ............................ RNN
Units in dense layer : ............................ 10
Number of layers     : ............................ 2
Dropout rate         : ............................ 0.0
Seed value           : ............................ 42
Nb of classes (logit): ............................ 3
Weights initializer  : ............................ truncated_normal
Batch normalization  : ............................ True


Cell type            : ............................ SGRU
Hidden units         : ............................ 128
Stacked cells        : ............................ 2
Time steps           : ............................ 224

Max steps= 200 / Number of EPOCH=6


In [6]:
p8_util_config.dict_adanet_config['adanet_nn_layer_config']

{'nn_type': 'RNN',
 'nn_dropout_rate': 0.0,
 'nn_batch_norm': True,
 'nn_dense_layer_num': 1,
 'nn_dense_unit_size': 10,
 'nn_logit_dimension': 3,
 'nn_optimizer': <tensorflow.python.training.adam.AdamOptimizer at 0x7fe48050a518>,
 'nn_seed': 42,
 'nn_initializer_name': 'truncated_normal',
 'nn_layer_config': {'rnn_layer_num': 2,
  'rnn_hidden_units': 128,
  'rnn_timesteps': 224,
  'rnn_cell_type': 'SGRU'}}

In [7]:
# Parameters that will be provided to model_fn
import p8_util_config
dict_adanet_nn_layer_config = p8_util_config.dict_adanet_config['adanet_nn_layer_config']
nn_type = dict_adanet_nn_layer_config['nn_type']
params = {'net_builder':oNNAdaNetBuilder, 'nn_type':nn_type}
if 'RNN' == nn_type :
    dict_nn_layer_config = dict_adanet_nn_layer_config['nn_layer_config']
    rnn_cell_type = dict_nn_layer_config['rnn_cell_type']
    params['rnn_cell_type'] = rnn_cell_type
    #params = {'net_builder':oNNAdaNetBuilder, 'nn_type':nn_type, 'rnn_cell_type':rnn_cell_type}
    
print("\n NN Type= {}".format(nn_type))
my_config = p8_util.make_config(nn_type,output_dir=OUTPUT_DIR, is_restored=False)  

classifier = tf.estimator.Estimator(model_fn=p8_util.my_model_fn, params=params, config=my_config)


 NN Type= RNN

*** make_config() : output dir= ./tmp/baseline/RNN
INFO:tensorflow:Using config: {'_model_dir': './tmp/baseline/RNN', '_tf_random_seed': 42, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fe48050a748>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [8]:
oNNAdaNetBuilder._nb_class
oNNAdaNetBuilder._dict_rnn_layer_config['rnn_hidden_units']

128

In [9]:
tuple_dimension

(672, 224)

In [10]:
import p8_util

batch_test = 20

train_spec=tf.estimator.TrainSpec(
        input_fn=p8_util.input_fn("train", x_train, y_train, p8_util_config.NUM_EPOCHS\
                          , batch_size=p8_util_config.BATCH_SIZE\
                          , feature_shape = tuple_dimension\
                          ),
        max_steps=p8_util_config.MAX_STEPS)

eval_spec=tf.estimator.EvalSpec(
        input_fn=p8_util.input_fn("test", x_test[:batch_test], y_test[:batch_test]\
                          , p8_util_config.NUM_EPOCHS\
                          , batch_size=p8_util_config.BATCH_SIZE\
                          , feature_shape = tuple_dimension\
                           ),
        steps=None,
        throttle_secs=1)

In [11]:
start_time = time.time()
results, _ = tf.estimator.train_and_evaluate(classifier, train_spec=train_spec, eval_spec=eval_spec)
end_time = time.time()

INFO:tensorflow:Not using Distribute Coordinator.
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 10 or save_checkpoints_secs None.

*** input_fn() : feature_shape= (672, 224) / Label shape= [3]
INFO:tensorflow:Calling model_fn.

*** my_model_fn() : input_layer shape= (?, 150528) / Labels shape= (?, 3)

*** my_model_fn() : logits shape= (?, 3) / labels shape= (?, 3)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./tmp/baseline/RNN/model.ckpt.

*** generator() : labels shape= (414, 3) / label values= [1. 0. 0.]
INFO:tensorflow:loss = 1.1017659, step = 1

*** generator() : labels sh

INFO:tensorflow:Saving checkpoints for 80 into ./tmp/baseline/RNN/model.ckpt.

*** input_fn() : feature_shape= (672, 224) / Label shape= [3]
feature_shape= (672, 224)
INFO:tensorflow:Calling model_fn.

*** my_model_fn() : input_layer shape= (?, 150528) / Labels shape= (?, 3)

*** my_model_fn() : logits shape= (?, 3) / labels shape= (?, 3)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-09-08:29:33
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/baseline/RNN/model.ckpt-80
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

*** generator() : labels shape= (20, 3) / label values= [0. 1. 0.]
INFO:tensorflow:Finished evaluation at 2019-05-09-08:29:48
INFO:tensorflow:Saving dict for global step 80: RNN_Eval_accuracy = 0.65, global_step = 80, loss = 0.76146185
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 80: ./tmp/baseline/RNN/model.ckpt-80

*** generator() : labels s

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 150: ./tmp/baseline/RNN/model.ckpt-150
INFO:tensorflow:Saving checkpoints for 160 into ./tmp/baseline/RNN/model.ckpt.

*** input_fn() : feature_shape= (672, 224) / Label shape= [3]
feature_shape= (672, 224)
INFO:tensorflow:Calling model_fn.

*** my_model_fn() : input_layer shape= (?, 150528) / Labels shape= (?, 3)

*** my_model_fn() : logits shape= (?, 3) / labels shape= (?, 3)
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-09-08:33:32
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./tmp/baseline/RNN/model.ckpt-160
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

*** generator() : labels shape= (20, 3) / label values= [0. 1. 0.]
INFO:tensorflow:Finished evaluation at 2019-05-09-08:33:41
INFO:tensorflow:Saving dict for global step 160: RNN_Eval_accuracy = 0.8, global_step = 160, loss = 0.6502778
INFO:tensorflow:Saving 

In [12]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("{}: {}".format(title, results[key]))



------------------------------------------------
Time (sec) 673.2258858680725
RNN_EVAL_ACCURACY: 0.800000011920929
LOSS: 0.7224235534667969
GLOBAL_STEP: 200


In [20]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6007 &'
        .format(OUTPUT_DIR)
    )


    get_ipython().system_raw('./assets/ngrok http 6007 &')